In [ ]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Lambda



In [ ]:
top_words = 10000
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=top_words)
data = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
max_review_length = 500
data = sequence.pad_sequences(data, maxlen=max_review_length)


In [ ]:
embedding_vector_length = 32



In [ ]:
# model = Sequential()
# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Fit ansamble
models = []
n_models = 5
n_samples = int(len(data) / n_models)

for i in range(0, n_models):
    test_x = data[i * n_samples:(i + 1) * n_samples]
    test_y = targets[i * n_samples:(i + 1) * n_samples]
    train_x = np.concatenate([data[:i * n_samples], data[(i + 1) * n_samples:]], axis=0)
    train_y = np.concatenate([targets[:i * n_samples], targets[(i + 1) * n_samples:]], axis=0)

    train_x = sequence.pad_sequences(train_x, maxlen=max_review_length)
    test_x = sequence.pad_sequences(test_x, maxlen=max_review_length)

    model = Sequential()
    model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64)
    model.save('model' + str(i) + '.h5')
    models.append(model)

Epoch 1/3
625/625 [==============================] - 16s 24ms/step - loss: 0.5126 - accuracy: 0.7167 - val_loss: 0.2639 - val_accuracy: 0.8964
Epoch 2/3
625/625 [==============================] - 14s 23ms/step - loss: 0.2114 - accuracy: 0.9187 - val_loss: 0.2684 - val_accuracy: 0.8890
Epoch 3/3
625/625 [==============================] - 15s 24ms/step - loss: 0.1609 - accuracy: 0.9436 - val_loss: 0.2631 - val_accuracy: 0.8943
Epoch 1/3
625/625 [==============================] - 16s 24ms/step - loss: 0.5325 - accuracy: 0.6856 - val_loss: 0.2717 - val_accuracy: 0.8919
Epoch 2/3
625/625 [==============================] - 14s 23ms/step - loss: 0.2144 - accuracy: 0.9196 - val_loss: 0.2645 - val_accuracy: 0.8924
Epoch 3/3
625/625 [==============================] - 14s 23ms/step - loss: 0.1751 - accuracy: 0.9372 - val_loss: 0.2823 - val_accuracy: 0.8923
Epoch 1/3
625/625 [==============================] - 16s 24ms/step - loss: 0.5426 - accuracy: 0.6764 - val_loss: 0.2573 - val_accuracy: 0.8975

In [ ]:
# evaluate ansample


def ans(_inp):
  res=[]
  for i in range(len(models)):
    outp = models[i].predict(_inp)
    res.append(outp)

  return sum(res)/len(res)




In [ ]:

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# H = model.fit(data, targets, validation_split=.2, epochs=3, batch_size=64)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 250, 128)          82432     
_________________________________________________________________
dropout (Dropout)            (None, 250, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)               